# Connect to Clipper

In [ ]:
# clipper_manager must be on your path:
import sys
import os
sys.path.append(os.path.abspath('../../management/'))
import clipper_manager as cm
# Change the username if necessary
user = "ubuntu"
# Set the path to the SSH key
key = ""
# Set the SSH host
host = ""
clipper = cm.Clipper(host, user, key)

# Load Cifar

__Set the location to the downloaded Cifar dataset:__

In [ ]:
cifar_loc = ""

In [ ]:
import cifar_utils
train_x, train_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_train.data", norm=True))
test_x, test_y = cifar_utils.filter_data(
    *cifar_utils.load_cifar(cifar_loc, cifar_filename="cifar_test.data", norm=True))

# Train Logistic Regression Model

In [ ]:
from sklearn import linear_model as lm 
def train_sklearn_model(m, train_x, train_y):
    m.fit(train_x, train_y)
    return m
lr_model = train_sklearn_model(lm.LogisticRegression(), train_x, train_y)
print("Logistic Regression test score: %f" % lr_model.score(test_x, test_y))

# Deploy Logistic Regression Model

In [ ]:
model_added = clipper.deploy_model(
    "sklearn_cifar",
    1,
    lr_model,
    # TODO: change test to latest
    "clipper/sklearn_cifar_container:test",
    ["cifar", "sklearn"],
    "doubles",
    num_containers=1
)
print("Model deploy successful? {success}".format(success=model_added))

# Load TensorFlow Model

There is a pre-trained TensorFlow model stored in the repo using [`git-lfs`](https://git-lfs.github.com/). Once you install `git-lfs`, you can pull the model with the command `git lfs pull`. If you don't want to deploy a TensorFlow model, you can skip this step.

In [ ]:
import os
import tensorflow as tf
import numpy as np
tf_cifar_model_path = os.path.abspath("tf_cifar_model/cifar10_model_full")
tf_session = tf.Session('', tf.Graph())
with tf_session.graph.as_default():
    saver = tf.train.import_meta_graph("%s.meta" % tf_cifar_model_path)
    saver.restore(tf_session, tf_cifar_model_path)

def tensorflow_score(session, test_x, test_y):
    """
    NOTE: This predict method expects pre-whitened (normalized) images
    """
    logits = session.run('softmax_logits:0',
                           feed_dict={'x:0': test_x})
    relevant_activations = logits[:, [cifar_utils.negative_class, cifar_utils.positive_class]]
    preds = np.argmax(relevant_activations, axis=1)
    preds[preds == 0] = -1.0
    return float(np.sum(preds == test_y)) / float(len(test_y))
print("TensorFlow CNN test score: %f" % tensorflow_score(tf_session, test_x, test_y))

# Deploy TensorFlow Model

In [ ]:
model_added = clipper.deploy_model(
    "tf_cifar",
    1,
    os.path.abspath("tf_cifar_model"),
    # TODO: change test to latest
    "clipper/tf_cifar_container:test",
    ["cifar", "tf"],
    "doubles",
    num_containers=1
)
print("Model deploy successful? {success}".format(success=model_added))

# Inspect Selection Policy

In [ ]:
app_name = "cifar_demo"
clipper.inspect_selection_policy(app_name, 0)

# Start feedback

In [ ]:
cifar_utils.enable_feedback(host, app_name, test_x, test_y, 70)

# Inspect Selection Policy Again

In [ ]:
clipper.inspect_selection_policy(app_name, 0)

# Inspect Clipper Metrics

In [ ]:
clipper.inspect_instance()